# Classical Balanced Network Theory

In this tutorial, we will follow up on the lecture on balanced networks. We will simulate large networks of neurons and show that they can naturally produce the kind of collective spiking activity widely observed throughout the mammalian cortex:

<figure style="margin: 2em;">
    <img src="img/renart2010.png" style="width: 300px;">
    <figcaption style="text-align: center; margin-top:0.5em;">Population activity in rat neocortex [Renart et al., Science 2010].</figcaption>
</figure>

The notebook follows a slow progression from:

- generation of Poisson processes, to provide a source of external (variable) input to neurons
- showing that despite variability in their input, LIF model neurons tend to spike _regularly_ (in contrast with empirical observations), _unless they receive balanced excitatory and inhibitory inputs_.
- simulation of a full balanced network, and analysis



In [ ]:
;;
#require "pkp"

open Owl

open Gp

In [ ]:
module A = Pkp.Balanced_net (* module alias to simplify outputs *)

open A

In [ ]:
let k = 100 (* number of (exc. or inh.) input synapses per neuron; cf. lecture *)

Before you start, have a quick look at the [documentation](https://pkp-neuro.github.io/pkp-tutorials/pkp/Pkp/Balanced_net/index.html) of the `Pkp.Balanced_net` module.

## 1. Poisson neurons → they will provide input to our network

Poisson neurons are “fake neurons” which simply emit action potentials following a “Poisson process“.

Create an array of `n` Poisson neurons firing at 5 Hz -- cf. `poisson` function. Here, `n` is set to `k`. Use `Array.init`, not `Array.make`.

In [ ]:
let n = k

In [ ]:
let input = ... (* your code here *)

Now create a “network” (a simple record ─ cf. `network` type), and simulate it for `duration = 3.0` s.

In [ ]:
(* your code here *)

Inspect the spiking behaviour of this input population, by plotting a spike raster using this function:

In [ ]:
let plot_raster population =
  let spikes = population |> Array.map spikes |> raster in
  let figure (module P : Plot) =
    P.plot
      (A spikes)
      ~style:"p pt 7 lc 8 ps 0.4"
      [ barebone
      ; borders [ `bottom ]
      ; offsets [ `bottom (`graph 0.1) ]
      ; xtics `auto
      ; xlabel "time [s]"
      ; ylabel "neurons"
      ]
  in
  Juplot.draw ~fmt:`svg ~size:(600, 300) figure

Now run the simulation over a longer duration (e.g. 50 seconds).

- Verify numerically that neurons fire an average of 5 spikes per second, as they should.
- Compute the Fano factor for each neuron. What is it, on average? What should it be?

In [ ]:
(* your code here *)

## 2. A single LIF neuron receiving Poisson input

Now, create a function `make_simple_net` of type `float → (neuron * network)`, which takes a weight parameter `w` and:
   1. creates a single LIF neuron (function `lif`)
   2. creates connections from `input` (the `n` Poisson neurons above) to the LIF neuron, with weight $w/k$ (function `all_to_all_connections`)
   3. creates a network with all those neurons and connections
   4. returns the pair (single LIF, network)

In [ ]:
let make_simple_net w = 
 (* your code here *)

Here is a helper function to plot the activity (voltage timecourse + spikes) of the LIF neuron:

In [ ]:
let plot_response ~duration x =
  let figure (module P : Plot) =
    P.plots
      [ plottable_voltage ~duration x; plottable_spikes x ]
      [ barebone
      ; xrange (0., duration)
      ; ytics `auto
      ; margins [ `bottom 0.2 ]
      ; set
          "object 1 rectangle from first 0, graph -0.1 rto first 0.2, graph -0.02 fs \
           solid 1.0 noborder fc rgb 'black' noclip"
      ]
  in
  Juplot.draw ~fmt:`svg ~size:(500, 200) figure

Using the above function, together with your `make_simple_net` and `simulate ~duration:1.0` functions, explore the behaviour of this mini network. Start with `w=5.0` and increase it until you find that the LIF neuron's voltage goes above threshold.

In [ ]:
(* your code here *)

How does the firing rate of the LIF neuron depend on `w`? Plot this dependence.

In [ ]:
(* your code here *)

When `w` is set to achieve a firing rate of approximately 5 Hz, what is the Fano factor of the spike count distribution assuming a “counting window” of 0.1 s?

## 3. Single neuron receiving balanced E and I inputs

Now we are going to simulate a single neuron, receiving:
1. excitatory input from `k` Poisson neurons (5 Hz), with weight $+w/\sqrt{K}$, and
2. inhibitory input from `k` Poisson neurons (5 Hz; another, independent set), with weight $-w/\sqrt{K}$

Begin by writing a function `make_simple_ei_net` of type `float → (neuron * network)` (similar to `make_simple_net` above) that sets up the whole network given the parameter `w`.

In [ ]:
let make_simple_ei_net w = (* your code here *)

Now, repeat the analysis of the previous section:
1. Plot the voltage+spike response of your LIF neuron, and explore the effect of `w`
2. Plot the firing rate of the LIF neuron as a function of `w`.
3. Find the Fano factor when `w` is set so as to achieve a firing rate of ~5 Hz.

You might want to reuse some of your previous code. 

In [ ]:
(* your code here *)

## 4. Full balanced network

We are now ready to simulate the full network.

To begin with, let's define a custom record type to hold all our weight parameters (will come in handy later):

In [ ]:
(* cf lecture slides: e.g. "ex" means "from x to e" *)
type weight =
  { ex : float
  ; ix : float
  ; ee : float
  ; ei : float
  ; ie : float
  ; ii : float
  }

This is a good set of default parameters you might want to use later:

In [ ]:
let default_weights =
  { ex = 1.0; ix = 1.0; ee = 1.0; ei = -2.0; ie = 1.0; ii = -1.8 }

Now, write a function of type `weights → (neuron array * neuron array * neuron array) * network` which:
1. creates a population of $N=1000$ Poisson neurons (5 Hz rate) ─ call this `popX`
2. creates a population of $N$ (excitatory) LIF neurons ─ call this `popE`; make sure that all but the very first neuron have `~log_voltage:false` in their options.
3. creates a population of $N$ (inhibitory) LIF neurons ─ call this `popI`; make sure that all but the very first neuron have `~log_voltage:false` in their options.
4. sets up random connections as discussed in the lecture: each neuron in each of the {X, E, and I} population makes a connection onto `k` randomly chosen neurons in both `popE` and `popI`. You will want to use the `random_connections` function provided in `module A`. Set the connection weights appropriately, don't forget the $1/\sqrt{K}$ factor!
5. return a tuple with the 3 populations, along with the full network

In [ ]:
let make_full_net weights = (* your code here *)

Here is a function that plots a summary of the network output.

In [ ]:
let plot_network_output ~duration (popE, popI, popX) =
  let keep = 100 in
  let popE = Array.sub popE 0 keep in
  let popI = Array.sub popI 0 keep in
  let popX = Array.sub popX 0 keep in
  let figure (module P : Plot) =
    let common =
      [ barebone; xrange (0.0, duration); margins [ `left 0.2; `right 0.95 ] ]
    in
    let plot_raster ~tm ~bm (pop, name, color) =
      P.plot
        (A (popX |> Array.map spikes |> raster))
        ~style:(Printf.sprintf "p pt 7 lc rgb '%s' ps 0.4" color)
        (common @ [ margins [ `top tm; `bottom bm ]; ylabel name ])
    in
    plot_raster ~tm:0.9 ~bm:0.7 (popX, "X neurons", "black");
    plot_raster ~tm:0.68 ~bm:0.48 (popE, "E neurons", "#e51e10");
    plot_raster ~tm:0.46 ~bm:0.26 (popI, "I neurons", "#56b4e9");
    P.plots
      [ plottable_voltage ~duration popE.(0); plottable_spikes popE.(0) ]
      (common
      @ [ margins [ `top 0.24; `bottom 0.1 ]
        ; borders [ `bottom ]
        ; xtics (`regular [ 0.; 1. ])
        ; offsets [ `bottom (`graph 0.1) ]
        ; xlabel "time"
        ; ylabel "V_m"
        ])
  in
  Juplot.draw ~fmt:`svg ~size:(400, 600) figure

Compute the mean firing rates in the E and I populations. Do they match the theoretical expectation (cf. lecture)?

Compute the Fano factors in each population. What is it, on average? Comment.